In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from datetime import timedelta

# from cartoframes import read_carto, to_carto, has_table, list_tables
# from cartoframes.auth import set_default_credentials, Credentials
# #from cartoframes.viz import Map, Layer, basemaps
# from cartoframes.data.clients import SQLClient
from shapely.geometry import Point, LineString
from zipfile import ZipFile

### New Trips Data

In [102]:
header = list(pd.read_csv('TripBulkReportTripsHeaders.csv',keep_default_na=True))
header.append('Unnamed 31')
header.append('Unnamed 32')
large_trip_df = pd.read_csv('new_trips_large.csv', keep_default_na=True, names=header)
small_trip_df = pd.read_csv('new_trips_subset.csv1', keep_default_na=True, names=header)


frames = [large_trip_df,small_trip_df]
trip_df = pd.concat(frames)

In [103]:
len(trip_df)

3476502

In [104]:
trip_df.columns

Index(['TripId', 'DeviceId', 'ProviderId', 'Mode', 'StartDate', 'StartWDay',
       'EndDate', 'EndWDay', 'StartLocLat', 'StartLocLon', 'EndLocLat',
       'EndLocLon', 'GeospatialType', 'ProviderType', 'ProviderDrivingProfile',
       'VehicleWeightClass', 'ProbeSourceType', 'OriginZoneName',
       'DestinationZoneName', 'EndpointType', 'TripMeanSpeedKph',
       'TripMaxSpeedKph', 'TripDistanceMeters', 'MovementType', 'OriginCbg',
       'DestCbg', 'StartTimezone', 'EndTimezone', 'WaypointFreqSec', 'StartQk',
       'EndQk', 'Unnamed 31', 'Unnamed 32'],
      dtype='object')

In [106]:
number_of_dup = len(trip_df[trip_df.duplicated(subset=list(trip_df.columns).remove('TripId')) == True])
number_of_fort = len(trip_df[(trip_df['OriginZoneName']!='Buc-ee\'s Giddings TX')& (trip_df['DestinationZoneName']!='Buc-ee\'s Giddings TX')])

print('Number of rows in original data:', len(trip_df))
print('Number of data in Fort Carson : ',number_of_fort)
print('Number of duplicated rows : ',number_of_dup)

Number of rows in original data: 3476502
Number of data in Fort Carson :  3476502
Number of duplicated rows :  0


In [107]:
trip_df = trip_df[(trip_df['OriginZoneName']!='Buc-ee\'s Giddings TX')& (trip_df['DestinationZoneName']!='Buc-ee\'s Giddings TX')]
# creating a geometry column 
trip_df['Startlocation'] = list(zip(trip_df.StartLocLat, trip_df.StartLocLon))
trip_df['Startlocation'] = trip_df['Startlocation'].apply(Point)

trip_df['Endlocation'] = list(zip(trip_df.EndLocLat, trip_df.EndLocLon))
trip_df['Endlocation'] = trip_df['Endlocation'].apply(Point)

trip_df= trip_df.drop(['Unnamed 31', 'Unnamed 32'], axis=1)

In [108]:
gdf = gpd.GeoDataFrame(trip_df)

In [109]:
gdf['MST_start_datetime'] = pd.to_datetime(gdf['StartDate']).apply(lambda x:x.astimezone('MST'))
gdf['MST_end_datetime'] = pd.to_datetime(gdf['EndDate']).apply(lambda x:x.astimezone('MST'))

gdf['MST_start_date'] = gdf['MST_start_datetime'].apply(lambda x:x.date())
gdf['MST_start_time'] = gdf['MST_start_datetime'].apply(lambda x:x.time())

gdf['MST_end_date'] = gdf['MST_end_datetime'].apply(lambda x:x.date())
gdf['MST_end_time'] = gdf['MST_end_datetime'].apply(lambda x:x.time())

In [110]:
# create df of start day, month, year 
startYear = list(pd.DatetimeIndex(gdf['MST_start_date']).year)
startMonth = list(pd.DatetimeIndex(gdf['MST_start_date']).month)
startDay = list(pd.DatetimeIndex(gdf['MST_start_date']).day)

# insert the start month, day, year column into the gdf
gdf.insert(int(gdf.columns.get_indexer(['MST_start_date'])[0]+1), 'MST_start_month',startMonth)
gdf.insert(int(gdf.columns.get_indexer(['MST_start_date'])[0]+2), 'MST_start_day',startDay)
gdf.insert(int(gdf.columns.get_indexer(['MST_start_date'])[0]+3), 'MST_start_year',startYear)

# create df of start hour, minutes, and seconds
startHour = list(pd.DatetimeIndex(gdf['MST_start_datetime']).hour)
startMin = list(pd.DatetimeIndex(gdf['MST_start_datetime']).minute)
startSec = list(pd.DatetimeIndex(gdf['MST_start_datetime']).second)

# insert the start hour, minutes, seconds column into the gdf
gdf.insert(int(gdf.columns.get_indexer(['MST_start_time'])[0]+1), 'MST_start_hour',startHour)
gdf.insert(int(gdf.columns.get_indexer(['MST_start_time'])[0]+2), 'MST_start_minute',startMin)
gdf.insert(int(gdf.columns.get_indexer(['MST_start_time'])[0]+3), 'MST_start_second',startSec)

# create df of end day, month, year 
endYear = list(pd.DatetimeIndex(gdf['MST_end_date']).year)
endMonth = list(pd.DatetimeIndex(gdf['MST_end_date']).month)
endDay = list(pd.DatetimeIndex(gdf['MST_end_date']).day)

# insert the start month, day, year column into the gdf
gdf.insert(int(gdf.columns.get_indexer(['MST_end_date'])[0]+1), 'MST_end_month',endMonth)
gdf.insert(int(gdf.columns.get_indexer(['MST_end_date'])[0]+2), 'MST_end_day',endDay)
gdf.insert(int(gdf.columns.get_indexer(['MST_end_date'])[0]+3), 'MST_end_year',endYear)

# create df of end hour, minutes, and seconds
endHour = list(pd.DatetimeIndex(gdf['MST_end_datetime']).hour)
endMin = list(pd.DatetimeIndex(gdf['MST_end_datetime']).minute)
endSec = list(pd.DatetimeIndex(gdf['MST_end_datetime']).second)


# insert the start hour, minutes, seconds column into the gdf
gdf.insert(int(gdf.columns.get_indexer(['MST_end_time'])[0]+1), 'MST_end_hour',endHour)
gdf.insert(int(gdf.columns.get_indexer(['MST_end_time'])[0]+2), 'MST_end_minute',endMin)
gdf.insert(int(gdf.columns.get_indexer(['MST_end_time'])[0]+3), 'MST_end_second',endSec)


In [111]:
gdf.columns

Index(['TripId', 'DeviceId', 'ProviderId', 'Mode', 'StartDate', 'StartWDay',
       'EndDate', 'EndWDay', 'StartLocLat', 'StartLocLon', 'EndLocLat',
       'EndLocLon', 'GeospatialType', 'ProviderType', 'ProviderDrivingProfile',
       'VehicleWeightClass', 'ProbeSourceType', 'OriginZoneName',
       'DestinationZoneName', 'EndpointType', 'TripMeanSpeedKph',
       'TripMaxSpeedKph', 'TripDistanceMeters', 'MovementType', 'OriginCbg',
       'DestCbg', 'StartTimezone', 'EndTimezone', 'WaypointFreqSec', 'StartQk',
       'EndQk', 'Startlocation', 'Endlocation', 'MST_start_datetime',
       'MST_end_datetime', 'MST_start_date', 'MST_start_month',
       'MST_start_day', 'MST_start_year', 'MST_start_time', 'MST_start_hour',
       'MST_start_minute', 'MST_start_second', 'MST_end_date', 'MST_end_month',
       'MST_end_day', 'MST_end_year', 'MST_end_time', 'MST_end_hour',
       'MST_end_minute', 'MST_end_second'],
      dtype='object')

In [112]:
gdf['MST_start_date'].min()

datetime.date(2020, 8, 31)

In [113]:
gdf['MST_start_date'].max()

datetime.date(2021, 9, 30)

In [114]:
gdf['MST_start_date'].nunique()

396

In [115]:
gdf['MST_start_datetime'].unique()

<DatetimeArray>
[       '2020-12-28 07:49:16-07:00',        '2020-12-22 13:15:06-07:00',
        '2020-12-21 18:32:52-07:00',        '2020-12-22 07:13:19-07:00',
        '2020-12-22 04:46:26-07:00',        '2020-12-28 15:44:28-07:00',
        '2020-12-22 08:37:10-07:00',        '2020-12-22 16:03:37-07:00',
        '2020-12-27 13:09:25-07:00',        '2020-12-23 18:09:33-07:00',
 ...
        '2020-09-08 15:09:43-07:00', '2020-09-08 18:02:16.688000-07:00',
        '2020-09-11 14:25:27-07:00',        '2020-09-14 13:45:10-07:00',
        '2020-09-13 22:40:10-07:00',        '2020-09-11 06:31:40-07:00',
        '2020-09-11 14:18:26-07:00',        '2020-09-14 11:15:53-07:00',
        '2020-09-08 19:09:58-07:00',        '2020-09-08 05:30:08-07:00']
Length: 3163325, dtype: datetime64[ns, MST]

In [11]:
len(gdf['TripId'].unique())

3476502

In [116]:
gdf['start_year_month'] = pd.to_datetime(gdf['MST_start_datetime']).dt.to_period('M')

/Users/cindyhuang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1101: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [117]:
gdf_202010_to_202009_new = gdf.loc[(gdf['start_year_month']!='2020-09') & (gdf['start_year_month']!='2020-08')]

In [119]:
gdf_202010_to_202009_new['day_of_week'] = gdf_202010_to_202009_new['MST_start_datetime'].dt.day_name()

<ipython-input-119-93688cc6b1d5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_202010_to_202009_new['day_of_week'] = gdf_202010_to_202009_new['MST_start_datetime'].dt.day_name()


In [121]:
# np.sort(gdf_202010_to_202009_new[pd.to_datetime(gdf_202010_to_202009_new['MST_start_date']).dt.day_name()=='Monday']['MST_start'].unique())

In [122]:
# gdf_202010_to_202009_new['MST_start_date'].nunique()

In [123]:
workday = gdf_202010_to_202009_new[(gdf_202010_to_202009_new['day_of_week']!='Saturday') &(gdf_202010_to_202009_new['day_of_week']!='Sunday') ]

In [124]:
workday['MST_start_date'].nunique()

261

In [125]:
number_of_EI_weekday = workday.groupby(by = ["GeospatialType"]).count()['TripId']['EI']
number_of_weekdays = len(workday['MST_start_date'].unique())
print('Total number of EI trips in weekdays:',number_of_EI_weekday)
print('Total number of weekdays:',number_of_weekdays)
print('On average, the number of EI trips per day on weekdays is: {:.1f}'.format(number_of_EI_weekday/number_of_weekdays))

Total number of EI trips in weekdays: 736212
Total number of weekdays: 261
On average, the number of EI trips per day on weekdays is: 2820.7


In [ ]:
print('Total number of trips between 2020/10 and 2021/09:',len(gdf_202010_to_202009_new))

In [ ]:
np.sort(workday.groupby(by='StartWDay')['MST_start_date'].unique().iloc[0])

In [126]:
# gdf_202010_to_202009_new['day_of_week'].unique()

In [46]:
# old_data_workday = old_large_202010_to_202009[(old_large_202010_to_202009['day_of_week']!='Saturday') &(old_large_202010_to_202009['day_of_week']!='Sunday') ]

In [127]:
print('Total number of Trips on weekday by each Geospatial Type :')
workday.groupby(by = ["GeospatialType"]).count()['TripId']

Total number of Trips on weekday by each Geospatial Type :


GeospatialType
EI     736212
IE     735982
II    1370871
Name: TripId, dtype: int64

In [128]:
number_of_EI_weekday = workday.groupby(by = ["GeospatialType"]).count()['TripId']['EI']
number_of_weekdays = len(workday['MST_start_date'].unique())
print('Total number of EI trips in weekdays:',number_of_EI_weekday)
print('Total number of weekdays:',number_of_weekdays)
print('On average, the number of EI trips per day on weekdays is: {:.1f}'.format(number_of_EI_weekday/number_of_weekdays))

Total number of EI trips in weekdays: 736212
Total number of weekdays: 261
On average, the number of EI trips per day on weekdays is: 2820.7


In [129]:
print('Number of days for each workday:')
workday.groupby(by='day_of_week')['MST_start_date'].nunique()

Number of days for each workday:


day_of_week
Friday       52
Monday       52
Thursday     53
Tuesday      52
Wednesday    52
Name: MST_start_date, dtype: int64

In [130]:
number_of_EI_each_weekday = workday.loc[workday['GeospatialType']=='EI'].groupby(['day_of_week']).count()['TripId']


number_of_days = workday.groupby(by='day_of_week')['MST_start_date'].nunique()

daily_weekday_average = pd.concat([number_of_EI_each_weekday,number_of_days],axis=1).set_index(np.array(['Mon','Tue','Wed','Thurs','Fri'])).\
rename(columns={'TripId':'number of trips','MST_start_date':'number of days'})
daily_weekday_average['average number of trips'] = round(daily_weekday_average['number of trips']/daily_weekday_average['number of days'],1)

In [132]:
# number_of_EI_each_weekday = old_data_workday.loc[old_data_workday['GeospatialType']=='EI'].groupby(['day_of_week']).count()['TripId']


# number_of_days = old_data_workday.groupby(by='day_of_week')['MST_start_date'].nunique()

# old_daily_weekday_average = pd.concat([number_of_EI_each_weekday,number_of_days],axis=1).set_index(np.array(['Mon','Tue','Wed','Thurs','Fri'])).\
# rename(columns={'TripId':'number of trips','MST_start_date':'number of days'})
# old_daily_weekday_average['average number of trips'] = round(old_daily_weekday_average['number of trips']/old_daily_weekday_average['number of days'],1)

In [133]:
# print('Old data daily weekly average:\n')
# daily_weekday_average

In [134]:
print('New data daily weekly average:\n')
daily_weekday_average

New data daily weekly average:



,number of trips,number of days,average number of trips
Mon,142505,52,2740.5
Tue,136365,52,2622.4
Wed,152999,53,2886.8
Thurs,149929,52,2883.2
Fri,154414,52,2969.5


In [177]:
# gdf_202010_to_202009_new.head()

## Old data investigation

In [81]:
header = list(pd.read_csv('TripBulkReportTripsHeaders.csv',keep_default_na=True))
header.append('Unnamed 31')
header.append('Unnamed 32')
trip_df = pd.read_csv('trips_large_old.csv', keep_default_na=True, names=header)
small_trips_df = pd.read_csv('trips_small_old.csv', keep_default_na=True, names=header)

frames = [trip_df,small_trips_df]
old_data_gdf = pd.concat(frames)

In [84]:
len(old_data_gdf)


3235625

In [45]:
len(small_trips_df)

1160372

In [44]:
len(old_data_gdf)

3476421

In [85]:
old_data_gdf['StartDate'].nunique()

2941464

In [86]:
old_data_gdf.columns

Index(['TripId', 'DeviceId', 'ProviderId', 'Mode', 'StartDate', 'StartWDay',
       'EndDate', 'EndWDay', 'StartLocLat', 'StartLocLon', 'EndLocLat',
       'EndLocLon', 'GeospatialType', 'ProviderType', 'ProviderDrivingProfile',
       'VehicleWeightClass', 'ProbeSourceType', 'OriginZoneName',
       'DestinationZoneName', 'EndpointType', 'TripMeanSpeedKph',
       'TripMaxSpeedKph', 'TripDistanceMeters', 'MovementType', 'OriginCbg',
       'DestCbg', 'StartTimezone', 'EndTimezone', 'WaypointFreqSec', 'StartQk',
       'EndQk', 'Unnamed 31', 'Unnamed 32'],
      dtype='object')

In [87]:
old_data_gdf['StartDate'].min()


'2020-10-01T00:00:09.000Z'

In [88]:
old_data_gdf['StartDate'].max()


'2021-09-30T23:59:58.000Z'

In [122]:
old_data_gdf.head()

,TripId,DeviceId,ProviderId,Mode,StartDate,StartWDay,EndDate,EndWDay,StartLocLat,StartLocLon,...,MovementType,OriginCbg,DestCbg,StartTimezone,EndTimezone,WaypointFreqSec,StartQk,EndQk,Unnamed 31,Unnamed 32
0,4e6d3859173a783b35f234126e7d84ab,0da5283cac35fa6de9a83477d1477377,d07e70efcfab08731a97e7b91be644de,1,2020-12-28T14:49:16.000Z,1,2020-12-28T15:53:22.000Z,1,38.71498,-104.70176,...,1,NaN,NaN,America/Denver,America/Denver,NaN,23101211002301310,23101210113102230,NaN,NaN
1,a9f9b922d560178ab3bd1b773fef437d,0ec14c96079ca0e5b6afd3d0a429cf49,42998cf32d552343bc8e460416382dca,1,2020-12-22T20:15:06.000Z,2,2020-12-22T20:26:22.000Z,2,38.74497,-104.79377,...,1,NaN,NaN,America/Denver,America/Denver,NaN,23101210113103011,23101211000023121,NaN,NaN
2,f6aec89e2dd3cd8fe5d09e45fc083de9,13d0412d930e3669ac09040c767f1097,42998cf32d552343bc8e460416382dca,1,2020-12-22T01:32:52.000Z,1,2020-12-22T01:59:33.000Z,1,38.76318,-104.80476,...,1,NaN,NaN,America/Denver,America/Denver,NaN,23101210111320233,23101210111320232,NaN,NaN
3,b87c20dbcc739db4ff64c100398437b7,1520526e3b9c71175574099508704611,d07e70efcfab08731a97e7b91be644de,1,2020-12-22T14:13:19.000Z,2,2020-12-22T14:30:36.000Z,2,38.69890,-104.68940,...,1,NaN,NaN,America/Denver,America/Denver,NaN,23101211002330133,23101210113101210,NaN,NaN
4,ea65f3a1f58f2431dcfe317be0bdd002,15c0082ebc282eaf14a6c80c67017142,66f041e16a60928b05a7e228a89c3799,1,2020-12-22T11:46:26.000Z,2,2020-12-22T12:03:42.000Z,2,38.75568,-104.80339,...,1,NaN,NaN,America/Denver,America/Denver,NaN,23101210111322320,23101210113121113,NaN,NaN


In [89]:
old_data_gdf = old_data_gdf[(old_data_gdf['OriginZoneName']!='Buc-ee\'s Giddings TX')& (old_data_gdf['DestinationZoneName']!='Buc-ee\'s Giddings TX')]
# creating a geometry column 
old_data_gdf['Startlocation'] = list(zip(old_data_gdf.StartLocLat, old_data_gdf.StartLocLon))
old_data_gdf['Startlocation'] = old_data_gdf['Startlocation'].apply(Point)

old_data_gdf['Endlocation'] = list(zip(old_data_gdf.EndLocLat, old_data_gdf.EndLocLon))
old_data_gdf['Endlocation'] = old_data_gdf['Endlocation'].apply(Point)

old_data_gdf= old_data_gdf.drop(['Unnamed 31', 'Unnamed 32'], axis=1)

In [90]:
old_data_gdf = gpd.GeoDataFrame(old_data_gdf)

In [91]:
old_data_gdf['MST_start_datetime'] = pd.to_datetime(old_data_gdf['StartDate']).apply(lambda x:x.astimezone('MST'))
old_data_gdf['MST_end_datetime'] = pd.to_datetime(old_data_gdf['EndDate']).apply(lambda x:x.astimezone('MST'))

old_data_gdf['MST_start_date'] = old_data_gdf['MST_start_datetime'].apply(lambda x:x.date())
old_data_gdf['MST_start_time'] = old_data_gdf['MST_start_datetime'].apply(lambda x:x.time())

old_data_gdf['MST_end_date'] = old_data_gdf['MST_end_datetime'].apply(lambda x:x.date())
old_data_gdf['MST_end_time'] = old_data_gdf['MST_end_datetime'].apply(lambda x:x.time())

In [92]:
old_data_gdf['day_of_week'] = old_data_gdf['MST_start_datetime'].dt.day_name()

In [135]:
old_data_gdf.columns

Index(['TripId', 'DeviceId', 'ProviderId', 'Mode', 'StartDate', 'StartWDay',
       'EndDate', 'EndWDay', 'StartLocLat', 'StartLocLon', 'EndLocLat',
       'EndLocLon', 'GeospatialType', 'ProviderType', 'ProviderDrivingProfile',
       'VehicleWeightClass', 'ProbeSourceType', 'OriginZoneName',
       'DestinationZoneName', 'EndpointType', 'TripMeanSpeedKph',
       'TripMaxSpeedKph', 'TripDistanceMeters', 'MovementType', 'OriginCbg',
       'DestCbg', 'StartTimezone', 'EndTimezone', 'WaypointFreqSec', 'StartQk',
       'EndQk', 'Startlocation', 'Endlocation', 'MST_start_datetime',
       'MST_end_datetime', 'MST_start_date', 'MST_start_time', 'MST_end_date',
       'MST_end_time', 'day_of_week', 'start_year_month'],
      dtype='object')

In [99]:
old_data_gdf['MST_start_date'].nunique()

366

In [97]:
old_data_gdf['start_year_month'] = pd.to_datetime(old_data_gdf['MST_start_datetime']).dt.to_period('M')


/Users/cindyhuang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:1101: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  warnings.warn(


In [95]:
# trip_df = trip_df[(trip_df['OriginZoneName']!='Buc-ee\'s Giddings TX')& (trip_df['DestinationZoneName']!='Buc-ee\'s Giddings TX')]
# # creating a geometry column 
# trip_df['Startlocation'] = list(zip(trip_df.StartLocLat, trip_df.StartLocLon))
# trip_df['Startlocation'] = trip_df['Startlocation'].apply(Point)

# trip_df['Endlocation'] = list(zip(trip_df.EndLocLat, trip_df.EndLocLon))
# trip_df['Endlocation'] = trip_df['Endlocation'].apply(Point)

# trip_df= trip_df.drop(['Unnamed 31', 'Unnamed 32'], axis=1)

# trip_gdf = gpd.GeoDataFrame(trip_df)

# large_trip_gdf = trip_gdf

# large_trip_gdf['MST_start_datetime'] = pd.to_datetime(large_trip_gdf['StartDate']).apply(lambda x:x.astimezone('MST'))
# large_trip_gdf['MST_end_datetime'] = pd.to_datetime(large_trip_gdf['EndDate']).apply(lambda x:x.astimezone('MST'))

# large_trip_gdf['MST_start_date'] = large_trip_gdf['MST_start_datetime'].apply(lambda x:x.date())
# large_trip_gdf['MST_start_time'] = large_trip_gdf['MST_start_datetime'].apply(lambda x:x.time())

# large_trip_gdf['MST_end_date'] = large_trip_gdf['MST_end_datetime'].apply(lambda x:x.date())
# large_trip_gdf['MST_end_time'] = large_trip_gdf['MST_end_datetime'].apply(lambda x:x.time())

# large_trip_gdf['day_of_week'] = large_trip_gdf['MST_start_datetime'].dt.day_name()



In [137]:
old_data_workday = old_data_gdf[(old_data_gdf['day_of_week']!='Saturday') &(old_data_gdf['day_of_week']!='Sunday') ]

In [138]:
old_data_workday['MST_start_date'].nunique()

262

In [136]:
# large_trip_gdf['start_year_month'] = pd.to_datetime(large_trip_gdf['MST_start_datetime']).dt.to_period('M')


In [ ]:
# small_trips_df = small_trips_df[(small_trips_df['OriginZoneName']!='Buc-ee\'s Giddings TX')& (small_trips_df['DestinationZoneName']!='Buc-ee\'s Giddings TX')]
# # creating a geometry column 
# small_trips_df['Startlocation'] = list(zip(small_trips_df.StartLocLat, small_trips_df.StartLocLon))
# small_trips_df['Startlocation'] = small_trips_df['Startlocation'].apply(Point)

# small_trips_df['Endlocation'] = list(zip(small_trips_df.EndLocLat, small_trips_df.EndLocLon))
# small_trips_df['Endlocation'] = small_trips_df['Endlocation'].apply(Point)

# small_trips_df= small_trips_df.drop(['Unnamed 31', 'Unnamed 32'], axis=1)

# small_trips_df = gpd.GeoDataFrame(small_trips_df)

# small_trip_gdf = small_trips_df

# small_trip_gdf['MST_start_datetime'] = pd.to_datetime(small_trip_gdf['StartDate']).apply(lambda x:x.astimezone('MST'))
# small_trip_gdf['MST_end_datetime'] = pd.to_datetime(small_trip_gdf['EndDate']).apply(lambda x:x.astimezone('MST'))

# small_trip_gdf['MST_start_date'] = small_trip_gdf['MST_start_datetime'].apply(lambda x:x.date())
# small_trip_gdf['MST_start_time'] = small_trip_gdf['MST_start_datetime'].apply(lambda x:x.time())

# small_trip_gdf['MST_end_date'] = small_trip_gdf['MST_end_datetime'].apply(lambda x:x.date())
# small_trip_gdf['MST_end_time'] = small_trip_gdf['MST_end_datetime'].apply(lambda x:x.time())

# small_trip_gdf['day_of_week'] = small_trip_gdf['MST_start_datetime'].dt.day_name()

In [ ]:
# small_trip_gdf['start_year_month'] = pd.to_datetime(small_trip_gdf['MST_start_datetime']).dt.to_period('M')


In [140]:
old_gdf_202010_to_202009 = old_data_gdf.loc[(old_data_gdf['start_year_month']!='2020-09') & (old_data_gdf['start_year_month']!='2020-08')]
start = old_gdf_202010_to_202009['MST_start_date'].min()
end = old_gdf_202010_to_202009['MST_start_date'].max()
start_new = gdf_202010_to_202009_new['MST_start_date'].min()
end_new = gdf_202010_to_202009_new['MST_start_date'].max()
print('Old trips aggregated dataset time range is between '+str(old_data_gdf['MST_start_date'].min())+ ' and '+str(gdf['MST_start_date'].max()))
print('\nNew trips dataset time range is between '+str(gdf['MST_start_date'].min())+ ' and '+str(gdf['MST_start_date'].max()))


print('Total numbers of trips in old aggredated data : '+str(len(old_gdf_202010_to_202009['TripId'].unique())))
print('Total numbers of trips in new data : '+str(len(gdf_202010_to_202009_new['TripId'].unique())))

Old trips aggregated dataset time range is between 2020-09-30 and 2021-09-30

New trips dataset time range is between 2020-08-31 and 2021-09-30
Total numbers of trips in old aggredated data : 3234574
Total numbers of trips in new data : 3234574


In [ ]:
# old_gdf_202010_to_202009 = old_data_gdf.loc[(old_data_gdf['start_year_month']!='2020-09') & (old_data_gdf['start_year_month']!='2020-08')]
# old_large_202010_to_202009 = large_trip_gdf.loc[(large_trip_gdf['start_year_month']!='2020-09') & (large_trip_gdf['start_year_month']!='2020-08')]
# old_small_202010_to_202009 = small_trip_gdf.loc[(small_trip_gdf['start_year_month']!='2020-09') & (small_trip_gdf['start_year_month']!='2020-08')]
# small_start = small_trip_gdf['MST_start_date'].min()
# small_end = small_trip_gdf['MST_start_date'].max()
# large_start = large_trip_gdf['MST_start_date'].min()
# large_end = large_trip_gdf['MST_start_date'].max()

# start = old_gdf_202010_to_202009['MST_start_date'].min()
# end = old_gdf_202010_to_202009['MST_start_date'].max()


# old_large_start = old_large_202010_to_202009['MST_start_date'].min()
# old_large_end= old_large_202010_to_202009['MST_start_date'].max()

# old_small_start = old_small_202010_to_202009['MST_start_date'].min()
# old_small_end= old_small_202010_to_202009['MST_start_date'].max()

# start_new = gdf_202010_to_202009_new['MST_start_date'].min()
# end_new = gdf_202010_to_202009_new['MST_start_date'].max()
# print('Old trips aggregated dataset time range is between '+str(old_data_gdf['MST_start_date'].min())+ ' and '+str(old_data_gdf['MST_start_date'].max()))
# # print('Old trips large dataset time range is between '+str(large_start)+ ' and '+str(large_end))
# # print('Old trips subset data time range is between '+str(small_start)+ ' and '+str(small_end))


# print('\nNew trips dataset time range is between '+str(gdf['MST_start_date'].min())+ ' and '+str(gdf['MST_start_date'].max()))
      

# print('\nSetting time range between 2020-10-01 and 2021-09-30, the total number of trips breakdown is as follows: ')

# # print('Old trips large dataset time range is between '+str(old_large_start)+ ' and '+str(old_large_end))
# # print('Old trips subset data time range is between '+str(old_small_start)+ ' and '+str(old_small_end))



# print('Total numbers of trips in old aggredated data : '+str(len(old_gdf_202010_to_202009['TripId'].unique())))
# print('Total numbers of trips in new data : '+str(len(gdf_202010_to_202009_new['TripId'].unique())))
# print('Total numbers of trips in old large data : '+str(len(old_large_202010_to_202009['TripId'].unique())))
# print('Total numbers of trips in old sub-data : '+str(len(old_small_202010_to_202009['TripId'].unique())))


In [142]:
old_gdf_202010_to_202009['weekday_or_weekend'] = old_gdf_202010_to_202009['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')

EI_gdf = old_gdf_202010_to_202009.loc[old_gdf_202010_to_202009['GeospatialType']=='EI']

total_number_of_EI_trips = len(EI_gdf)
number_of_days = len(old_gdf_202010_to_202009['MST_start_date'].unique())

EI_trip_weekday_weekend = EI_gdf.groupby(by=['weekday_or_weekend']).count()['TripId']

number_of_weekday_weekend = EI_gdf.groupby(by=['weekday_or_weekend'])['MST_start_date'].nunique()

weekend_weekday_avg = pd.concat([EI_trip_weekday_weekend,number_of_weekday_weekend],axis=1)\
.rename(columns={'TripId':'number of EI trips','MST_start_date':'number of days'})
weekend_weekday_avg['avg number of trips'] = round(weekend_weekday_avg['number of EI trips']/weekend_weekday_avg['number of days'],1)
weekend_weekday_avg['number of EI trips'].apply(lambda x : "{:,}".format(x))
print('EI trips breakdown on old trip data with range between '+str(start)+ ' and '+str(end))

weekend_weekday_avg

weekday_avg = weekend_weekday_avg.loc['Weekday']['avg number of trips']
weekend_avg = weekend_weekday_avg.loc['Weekend']['avg number of trips']
print('On average, there are {:,.0f} trips onto and off of Fort Carson per day'.format(total_number_of_EI_trips/number_of_days))
print('On average, there are {:,.0f} trips onto and off of Fort Carson per day for weekdays'.format(weekday_avg))
print('On average, there are {:,.0f} trips onto and off of Fort Carson per day for weekends'.format(weekend_avg))



<ipython-input-142-2c705aecae7f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old_gdf_202010_to_202009['weekday_or_weekend'] = old_gdf_202010_to_202009['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')


EI trips breakdown on old trip data with range between 2020-10-01 and 2021-09-30
On average, there are 2,380 trips onto and off of Fort Carson per day
On average, there are 2,821 trips onto and off of Fort Carson per day for weekdays
On average, there are 1,275 trips onto and off of Fort Carson per day for weekends


In [143]:
gdf_202010_to_202009_new['weekday_or_weekend'] = gdf_202010_to_202009_new['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')

EI_gdf = gdf_202010_to_202009_new.loc[gdf_202010_to_202009_new['GeospatialType']=='EI']

total_number_of_EI_trips = len(EI_gdf)
number_of_days = len(gdf_202010_to_202009_new['MST_start_date'].unique())

# old_data_gdf['weekday_or_weekend'] = old_data_gdf['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')

EI_trip_weekday_weekend = EI_gdf.groupby(by=['weekday_or_weekend']).count()['TripId']

number_of_weekday_weekend = EI_gdf.groupby(by=['weekday_or_weekend'])['MST_start_date'].nunique()

weekend_weekday_avg = pd.concat([EI_trip_weekday_weekend,number_of_weekday_weekend],axis=1)\
.rename(columns={'TripId':'number of EI trips','MST_start_date':'number of days'})
weekend_weekday_avg['avg number of trips'] = round(weekend_weekday_avg['number of EI trips']/weekend_weekday_avg['number of days'],1)
weekend_weekday_avg['number of EI trips'].apply(lambda x : "{:,}".format(x))
print('EI trips breakdown on new trip data with range between '+str(start_new)+ ' and '+str(end_new))
weekend_weekday_avg

weekday_avg = weekend_weekday_avg.loc['Weekday']['avg number of trips']
weekend_avg = weekend_weekday_avg.loc['Weekend']['avg number of trips']
print('On average, there are {:,.0f} trips onto and off of Fort Carson per day'.format(total_number_of_EI_trips/number_of_days))
print('On average, there are {:,.0f} trips onto and off of Fort Carson per day for weekdays'.format(weekday_avg))
print('On average, there are {:,.0f} trips onto and off of Fort Carson per day for weekends'.format(weekend_avg))

<ipython-input-143-d904278541bf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf_202010_to_202009_new['weekday_or_weekend'] = gdf_202010_to_202009_new['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')


EI trips breakdown on new trip data with range between 2020-10-01 and 2021-09-30
On average, there are 2,380 trips onto and off of Fort Carson per day
On average, there are 2,821 trips onto and off of Fort Carson per day for weekdays
On average, there are 1,275 trips onto and off of Fort Carson per day for weekends


In [29]:
gdf_202010_to_202009_new.columns

gdf_202010_to_202009_new['MST_start_date'].nunique()


365

In [144]:
workday = gdf_202010_to_202009_new[(gdf_202010_to_202009_new['day_of_week']!='Saturday') &(gdf_202010_to_202009_new['day_of_week']!='Sunday') ]

In [145]:
print('Total number of Each Type of Trips in weekday on new data:')
workday.groupby(by = ["GeospatialType"]).count()['TripId']

Total number of Each Type of Trips in weekday on new data:


GeospatialType
EI     736212
IE     735982
II    1370871
Name: TripId, dtype: int64

In [146]:
print('Total number of Each Type of Trips in weekday on old data:')
old_data_workday.groupby(by = ["GeospatialType"]).count()['TripId']

Total number of Each Type of Trips in weekday on old data:


GeospatialType
EI     736509
IE     736361
II    1371246
Name: TripId, dtype: int64

In [147]:
number_of_EI_weekday = old_data_workday.groupby(by = ["GeospatialType"]).count()['TripId']['EI']
number_of_weekdays = len(old_data_workday['MST_start_date'].unique())
print('EI trip break down of old data between 2020-10-01 and 2021-09-30:\n ')
print('Total number of EI trips in weekdays:',number_of_EI_weekday)
print('Total number of weekdays:',number_of_weekdays)
print('\nOn average, the number of EI trips per day on weekdays is: {:.1f}'.format(number_of_EI_weekday/number_of_weekdays))

print('Number of days for each workday:')
old_data_workday.groupby(by='day_of_week')['MST_start_date'].nunique()

EI trip break down of old data between 2020-10-01 and 2021-09-30:
 
Total number of EI trips in weekdays: 736509
Total number of weekdays: 262

On average, the number of EI trips per day on weekdays is: 2811.1
Number of days for each workday:


day_of_week
Friday       52
Monday       52
Thursday     53
Tuesday      52
Wednesday    53
Name: MST_start_date, dtype: int64

In [148]:
number_of_EI_weekday = workday.groupby(by = ["GeospatialType"]).count()['TripId']['EI']
number_of_weekdays = len(workday['MST_start_date'].unique())
print('EI trip break down of new data between 2020-10-01 and 2021-09-30\n ')
print('Total number of EI trips in weekdays:',number_of_EI_weekday)
print('Total number of weekdays:',number_of_weekdays)
print('\nOn average, the number of EI trips per day on weekdays is: {:.1f}'.format(number_of_EI_weekday/number_of_weekdays))

print('Number of days for each workday:')
workday.groupby(by='day_of_week')['MST_start_date'].nunique()

EI trip break down of new data between 2020-10-01 and 2021-09-30
 
Total number of EI trips in weekdays: 736212
Total number of weekdays: 261

On average, the number of EI trips per day on weekdays is: 2820.7
Number of days for each workday:


day_of_week
Friday       52
Monday       52
Thursday     53
Tuesday      52
Wednesday    52
Name: MST_start_date, dtype: int64

In [150]:
old_data_gdf['weekday_or_weekend'] = old_data_gdf['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')

EI_gdf = old_data_gdf.loc[old_data_gdf['GeospatialType']=='EI']

total_number_of_EI_trips = len(EI_gdf)
number_of_days = len(old_data_gdf['MST_start_date'].unique())

# old_data_gdf['weekday_or_weekend'] = old_data_gdf['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')

EI_trip_weekday_weekend = EI_gdf.groupby(by=['weekday_or_weekend']).count()['TripId']

number_of_weekday_weekend = EI_gdf.groupby(by=['weekday_or_weekend'])['MST_start_date'].nunique()

weekend_weekday_avg = pd.concat([EI_trip_weekday_weekend,number_of_weekday_weekend],axis=1)\
.rename(columns={'TripId':'number of EI trips','MST_start_date':'number of days'})
weekend_weekday_avg['avg number of trips'] = round(weekend_weekday_avg['number of EI trips']/weekend_weekday_avg['number of days'],1)
weekend_weekday_avg['number of EI trips'].apply(lambda x : "{:,}".format(x))
print('EI trip breakdown by weekday/weekend for old data:\n ')

weekend_weekday_avg


EI trip breakdown by weekday/weekend for old data:
 


,number of EI trips,number of days,avg number of trips
weekday_or_weekend,,,
Weekday,736509,262,2811.1
Weekend,132612,104,1275.1


In [167]:
gdf['weekday_or_weekend'] = gdf['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')

EI_gdf = gdf.loc[gdf['GeospatialType']=='EI']

total_number_of_EI_trips = len(EI_gdf)
number_of_days = len(gdf['MST_start_date'].unique())

# old_data_gdf['weekday_or_weekend'] = old_data_gdf['day_of_week'].apply(lambda x:'Weekend' if (x=='Saturday' or x=='Sunday')else 'Weekday')

EI_trip_weekday_weekend = EI_gdf.groupby(by=['weekday_or_weekend']).count()['TripId']

number_of_weekday_weekend = EI_gdf.groupby(by=['weekday_or_weekend'])['MST_start_date'].nunique()

weekend_weekday_avg = pd.concat([EI_trip_weekday_weekend,number_of_weekday_weekend],axis=1)\
.rename(columns={'TripId':'number of EI trips','MST_start_date':'number of days'})
weekend_weekday_avg['avg number of trips'] = round(weekend_weekday_avg['number of EI trips']/weekend_weekday_avg['number of days'],1)
weekend_weekday_avg['number of EI trips'].apply(lambda x : "{:,}".format(x))
print('EI trip breakdown by weekday/weekend for new data:\n ')

weekend_weekday_avg

EI trip breakdown by weekday/weekend for new data:
 


,number of EI trips,number of days,avg number of trips
weekday_or_weekend,,,
Weekday,528658,284,1861.5
Weekend,94681,112,845.4


In [181]:
# start = old_data_gdf['MST_start_date'].min()
# end = old_data_gdf['MST_start_date'].max()
# start_new = gdf_202010_to_202009_new['MST_start_date'].min()
# end_new = gdf_202010_to_202009_new['MST_start_date'].max()
# print('Old trips dataset time range is between '+str(start)+ ' and '+str(end))
# print('New trips dataset time range is between '+str(start_new)+ ' and '+str(end_new))


In [180]:
# print('Total number of trips in old data is ' +str(len(old_gdf_202010_to_202009['TripId'].unique())))
# print('Total number of trips in new data is ' +str(len(gdf_202010_to_202009_new['TripId'].unique())))


In [92]:
# gdf_20200930 = old_data_gdf[old_data_gdf['MST_start_date']== '2020-09-30']

In [179]:
# len(gdf_20200930['TripId'].unique())

In [151]:
# trip_df = pd.read_csv('trips (1).csv', keep_default_na=True, names=header)
# small_trips_df = pd.read_csv('trips.csv1.csv', keep_default_na=True, names=header)

In [152]:
# print('large data total number of trips: '+ str(len(trip_df['TripId'].unique())))
# print('small data total number of trips: '+ str(len(small_trips_df['TripId'].unique())))

# test = len(trip_df) + 1051

In [ ]:
# trip_df['Startlocation'] = list(zip(trip_df.StartLocLat, trip_df.StartLocLon))
# trip_df['Startlocation'] = trip_df['Startlocation'].apply(Point)

# trip_df['Endlocation'] = list(zip(trip_df.EndLocLat, trip_df.EndLocLon))
# trip_df['Endlocation'] = trip_df['Endlocation'].apply(Point)

# trip_df= trip_df.drop(['Unnamed 31', 'Unnamed 32'], axis=1)

# large_gdf = gpd.GeoDataFrame(trip_df)
# large_gdf['MST_start_datetime'] = pd.to_datetime(large_gdf['StartDate']).apply(lambda x:x.astimezone('MST'))
# large_gdf['MST_end_datetime'] = pd.to_datetime(large_gdf['EndDate']).apply(lambda x:x.astimezone('MST'))

# large_gdf['MST_start_date'] = large_gdf['MST_start_datetime'].apply(lambda x:x.date())
# large_gdf['MST_start_time'] = large_gdf['MST_start_datetime'].apply(lambda x:x.time())

# large_gdf['MST_end_date'] = large_gdf['MST_end_datetime'].apply(lambda x:x.date())
# large_gdf['MST_end_time'] = large_gdf['MST_end_datetime'].apply(lambda x:x.time())

In [117]:
# small_trips_df['Startlocation'] = list(zip(trip_df.StartLocLat, small_trips_df.StartLocLon))
# small_trips_df['Startlocation'] = small_trips_df['Startlocation'].apply(Point)

# small_trips_df['Endlocation'] = list(zip(small_trips_df.EndLocLat, small_trips_df.EndLocLon))
# small_trips_df['Endlocation'] = small_trips_df['Endlocation'].apply(Point)

# # small_trips_df= small_trips_df.drop(['Unnamed 31', 'Unnamed 32'], axis=1)

# small_gdf = gpd.GeoDataFrame(small_trips_df)
# small_gdf['MST_start_datetime'] = pd.to_datetime(small_gdf['StartDate']).apply(lambda x:x.astimezone('MST'))
# small_gdf['MST_end_datetime'] = pd.to_datetime(small_gdf['EndDate']).apply(lambda x:x.astimezone('MST'))

# small_gdf['MST_start_date'] = small_gdf['MST_start_datetime'].apply(lambda x:x.date())
# small_gdf['MST_start_time'] = small_gdf['MST_start_datetime'].apply(lambda x:x.time())

# small_gdf['MST_end_date'] = small_gdf['MST_end_datetime'].apply(lambda x:x.date())
# small_gdf['MST_end_time'] = small_gdf['MST_end_datetime'].apply(lambda x:x.time())

In [153]:
print(old_data_gdf['MST_start_date'].min(), old_data_gdf['MST_start_date'].max())

2020-09-30 2021-09-30


### End of latest edit